# Tokenizer
This code aims to create a tokenizer and embed data from the "the-verdict.txt" text file. The source for this exercise is from: https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb

First, we import the text file and look at how many characters we have. 

In [8]:
if not os.path.exists("the-verdict.txt"):
       url = ("https://raw.githubusercontent.com/rasbt/"
              "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
              "the-verdict.txt")
       file_path = "the-verdict.txt"
       urllib.request.urlretrieve(url, file_path)

In [9]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Tokenizing text
We use regex to split on commas, full stops, and spaces, converting into an array.

In [22]:
text = "Hello, world. This, is a test"
result = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test']


Next, we add all other types of punctuation into our regex expression.

In [25]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


After texting, we pre-process our chosen file for embedding.

In [27]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [28]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


We then proceed to assign every token in the vocabulary a unique integer ID.

In [30]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50: break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


Comvining all of these code chunks, we create a tokenizer class, which can both encode and decode data.

In [33]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

We then test this on some sample sentnences

In [34]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [35]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [36]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## Special Context Tokens
We add in special context tokens to help the LLM with additional context, an unknown token and a token to signify the end of the text.

In [38]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<unk>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
len(vocab.items())

1132

In [39]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

## BytePair Encoding
GPT-2 uses a BPE tokenizer, which breaks down words that aret in the vocabulary into sub-words and individual characters, which allows it to handle out-of-vocabulary words.

In [40]:
import tiktoken

In [41]:
tokenizer = tiktoken.get_encoding("gpt2")

In [42]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [43]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


## Sliding window
We train LLMs to generate one word at a time, so we need to prepare the data accordingly,where the next word in the sequence represents the prediction target. We do this by sampling with a context window.

In [44]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [48]:
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x:{x}")
print(f"y:     {y}")

x:[290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [49]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [50]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))
    

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [52]:
import torch
print(torch.__version__)

2.8.0


We use the torch library and the sliding window concept to extract chunks from the input text dataset. 

In [62]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokens must be at least equal to max_length +1 "
        
        #sliding window
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i + max_length +1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [63]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle = shuffle, drop_last = drop_last, num_workers=num_workers)
    return dataloader

In [64]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [65]:
dataloader = create_dataloader_v1( raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [66]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


We can also create batched outputs. We increase strides to prevent overfitting. 

In [67]:
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = 4, shuffle = False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs: \n", inputs)
print("\nTargetrs:\n", targets)

Inputs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targetrs:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Creating token embeddings & positional encodings.
The data is now almost ready for use - but needs to emebedded in a continuous vector representation using an embedding layer. We also need to add positional endcodings to represent where a token is located in a sequeunce. 

In [68]:
vocab_size = 50258
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [69]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length=max_length, stride = max_length, shuffle = False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [70]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [71]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[-4.0923e-01, -1.2252e+00,  5.9993e-01,  ..., -1.1078e-01,
          -1.1088e+00,  1.1405e+00],
         [ 2.3548e+00, -7.6591e-01, -7.5249e-01,  ...,  1.6080e+00,
          -2.8281e-01,  2.5632e+00],
         [ 7.5632e-02,  1.3117e-01, -1.4037e+00,  ..., -4.0014e-01,
           8.6878e-01,  6.6945e-03],
         [-7.1496e-01,  6.1019e-01, -9.9995e-01,  ...,  4.0882e-01,
          -2.6238e-01,  2.2496e+00]],

        [[ 1.7183e+00,  4.8768e-01,  1.8258e+00,  ...,  7.1086e-01,
           4.7555e-01, -8.0010e-01],
         [-1.7964e+00, -8.0186e-01, -1.3298e-01,  ..., -1.6305e-01,
           1.9982e-01, -7.2233e-01],
         [ 1.2059e+00, -5.2811e-03, -1.7531e+00,  ...,  8.1180e-01,
          -1.4962e+00,  2.0218e+00],
         [-5.5843e-02, -7.6114e-01,  6.4291e-01,  ...,  3.8262e-01,
           1.5706e-01,  1.3514e+00]],

        [[-1.4949e-01,  2.4897e-01,  3.3719e-01,  ...,  1.3890e+00,
           9.8496e-01, -3.4277e-01],
         [-9.4834e-01, -1.5

In [72]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 9.2901e-01,  2.4738e-01, -3.5620e-01,  ..., -2.7161e-01,
         -1.4775e+00,  2.2138e+00],
        [ 8.2973e-02, -3.8073e-01, -1.2307e-01,  ...,  1.6274e+00,
         -5.5543e-02,  1.5420e+00],
        [-1.0777e+00,  1.1805e+00, -2.1549e-03,  ..., -1.2508e+00,
         -1.8156e+00,  2.5404e+00],
        [-7.1613e-01, -1.6032e+00,  6.6653e-01,  ...,  2.8427e-01,
          6.6459e-01, -1.2241e-01]], requires_grad=True)


In [76]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 9.2901e-01,  2.4738e-01, -3.5620e-01,  ..., -2.7161e-01,
         -1.4775e+00,  2.2138e+00],
        [ 8.2973e-02, -3.8073e-01, -1.2307e-01,  ...,  1.6274e+00,
         -5.5543e-02,  1.5420e+00],
        [-1.0777e+00,  1.1805e+00, -2.1549e-03,  ..., -1.2508e+00,
         -1.8156e+00,  2.5404e+00],
        [-7.1613e-01, -1.6032e+00,  6.6653e-01,  ...,  2.8427e-01,
          6.6459e-01, -1.2241e-01]], grad_fn=<EmbeddingBackward0>)


In [77]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.5198, -0.9778,  0.2437,  ..., -0.3824, -2.5863,  3.3543],
         [ 2.4377, -1.1466, -0.8756,  ...,  3.2354, -0.3384,  4.1051],
         [-1.0020,  1.3116, -1.4058,  ..., -1.6509, -0.9468,  2.5470],
         [-1.4311, -0.9930, -0.3334,  ...,  0.6931,  0.4022,  2.1272]],

        [[ 2.6473,  0.7351,  1.4696,  ...,  0.4392, -1.0019,  1.4137],
         [-1.7134, -1.1826, -0.2561,  ...,  1.4644,  0.1443,  0.8196],
         [ 0.1282,  1.1752, -1.7552,  ..., -0.4390, -3.3119,  4.5621],
         [-0.7720, -2.3643,  1.3094,  ...,  0.6669,  0.8216,  1.2290]],

        [[ 0.7795,  0.4964, -0.0190,  ...,  1.1174, -0.4925,  1.8710],
         [-0.8654, -0.5333, -1.5266,  ..., -0.9984, -0.6695,  1.3913],
         [-1.2925,  1.6390, -0.0752,  ...,  0.3167, -2.0194,  3.0350],
         [-0.4866, -1.2527,  0.9982,  ..., -1.4181,  1.1414, -1.9132]],

        ...,

        [[ 3.1530,  0.5787, -0.4572,  ...,  0.8042, -1.2463,  2.5303],
         [-0.5072, -0.8759, -0.85